### Library Imports

In [1]:
import pandas as pd
import xarray as xr
import numpy as np

### Data imports

In [2]:
# Data info : 
nb_samples = 4 # 4 samples per year if 2050 ; 1 sample for 2020 (full year)

#filename= 'france_2020_elec_data_sorted.xlsx'
filename= 'france_2050_elec_data_sorted.xlsx'

# Testing : 
#filename = 'France_mix_test.xlsx'

In [3]:
elec_sheets = {}

for sample in range(nb_samples) : 
    year_day_sample = pd.DataFrame(pd.read_excel(filename,index_col = 0,sheet_name = sample))
    # Save, from the get-go, the possible years (needs more work to be fail proof)
    years = year_day_sample['YEAR'].unique()
    Hours = year_day_sample.index.unique().tolist()
    ###############################################
    #date2_year = year_day_sample['DAY'].unique()[:-1]
    ###############################################
    day_samples = year_day_sample.groupby('YEAR')

    # Add a dictionary for years
    elec_sheets[sample] = {}

    # Split years from the sheets    
    for year,data in day_samples :
        data.drop(columns = 'YEAR',inplace = True) 

        ############################################
        # Add a dictionary for days
        #elec_sheets[sample][year] = {}   
        #individual_days = data.groupby('DAY')
        #for day, day_ds in individual_days : 
        #    day_ds.drop(columns = 'DAY', inplace =True)
        #    elec_sheets[sample][year][day] =day_ds            
        #############################################

        # Store both years according to number of samples : 
        elec_sheets[sample][year]= data
   
#print(years,'\n',date2_year,'\n',Hours)
##################################################
#display(elec_sheets[0][years[0]][date2_year[0]])
##################################################
display(elec_sheets[0][years[0]])

,Nucleaire,Eolien en mer,Eolien terrestre,Centrales gaz,Solaire,Hydraulique,Autre renouvelables,Fioul,Charbon
Heures,,,,,,,,,
0,0.031248,0.161761,0.529757,0.027222,0.000000,0.228544,0.021467,0,0
1,0.031434,0.162436,0.530306,0.026771,0.000000,0.227426,0.021627,0,0
2,0.031611,0.163069,0.530928,0.026325,0.000000,0.226292,0.021775,0,0
3,0.031761,0.163837,0.531361,0.026083,0.000000,0.225100,0.021858,0,0
4,0.031849,0.163881,0.531003,0.025824,0.000000,0.225510,0.021934,0,0
5,0.031936,0.164519,0.532088,0.025852,0.000442,0.223150,0.022014,0,0
6,0.031323,0.161299,0.520459,0.025806,0.027972,0.211565,0.021576,0,0
7,0.028249,0.145344,0.462465,0.022772,0.144992,0.176733,0.019445,0,0
8,0.023851,0.123711,0.381008,0.016497,0.304077,0.134270,0.016586,0,0


In [4]:
# List the dimensions
#Hours =  elec_sheets[0][years[0]].index.tolist()

# Technologies : 
cols2keep = []

################################################
#cols2keep =elec_sheets[sheet_id][year][day].columns.to_list()
################################################

for sheet_id in elec_sheets.keys() :
    for year in years : # redundant, but usefull to make sure nothing is off according to years
        cols = elec_sheets[sheet_id][years[0]].columns.to_list()   
        cols2keep.extend(cols)

technologies = [x for x in set(cols2keep)]

# Adapt dataframes in case some technologies are missing at this timepoint : 
for sheet_id in elec_sheets.keys() :
    
     for year in years : 
         # Find missing columns
         cols_to_add = [x for x in technologies if x not in elec_sheets[sheet_id][year].columns]

         # Add the columns (if needed) : 
         for col in cols_to_add :
             print(col)
             elec_sheets[sheet_id][year][col] = 0
     
         # While there, also make sure if a column is partial, just add zeros
         elec_sheets[sheet_id][year] = elec_sheets[sheet_id][year].fillna(0)

         # And also : relabel according to the technologies list (to match when the xarray is built)
         elec_sheets[sheet_id][year]= elec_sheets[sheet_id][year][technologies]
        
        # Quick check (optional)
        # print(year)
        # display(elec_sheets[sheet_id][year].head(5))

###########################################
# In 2020, there is still a need to ensure the list of technologies match with the dataset when sent to xr
#for shee_id in elec_sheets.keys() : 
#    for year in years : 
#        for day in date2_year : 
#            elec_sheets[sheet_id][year][day]= elec_sheets[sheet_id][year][day][technologies]
############################################

# Number of snapshots available for the year
date2_year = [1]
for value in range(nb_samples): 
    date2_year.append(round((value+1)*365/nb_samples))

# Overview
print(Hours)
print(technologies)
print(date2_year)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
['Autre renouvelables', 'Centrales gaz', 'Solaire', 'Eolien en mer', 'Hydraulique', 'Charbon', 'Fioul', 'Eolien terrestre', 'Nucleaire']
[1, 91, 182, 274, 365]


In [5]:
# list the dataframes that need to be uploaded to xarray
yearly_data = []
################
#daily_data = []
################

# First by year
for year in years : 
     # Then by sample number (season here)
     data = [elec_sheets[sheet_id][year] for sheet_id in range(nb_samples)]
    
     # Append at the end the first entry (coming full circle) 
     data.append(elec_sheets[0][year])
     yearly_data.append(data)

    ################################################
    #for day in date2_year : 
    #    data = [elec_sheets[sheet_id][year][day] for sheet_id in range(nb_samples)]

    #    daily_data.append(data)
    ################################################



In [6]:
# Creating the xarray DataArray
elec_mixes = xr.DataArray(
    yearly_data,
    ##################
    #daily_data,
    #coords=[date2_year,years,Hours, technologies],
    #dims=["Date","year", "Hour","technology"]   
    ##################
    coords=[years,date2_year,Hours, technologies],
    dims=["year","Date", "Hour","technology"] 
)
display(elec_mixes)

<xarray.DataArray (year: 1, Date: 5, Hour: 24, technology: 9)>
array([[[[2.14674734e-02, 2.72224895e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.29756620e-01, 3.12478883e-02],
         [2.16274136e-02, 2.67710858e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.30305557e-01, 3.14344781e-02],
         [2.17746256e-02, 2.63253379e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.30927608e-01, 3.16114221e-02],
         ...,
         [2.05329347e-02, 3.32736952e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.20823894e-01, 2.99901394e-02],
         [2.09658786e-02, 2.90864444e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.25130959e-01, 3.04531822e-02],
         [2.12323267e-02, 2.70951131e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.29050008e-01, 3.08084822e-02]],

        [[2.57386956e-02, 3.48640549e-03, 0.00000000e+00, ...,
          0.00000000e+00, 5.16240567e-01, 3.08918672e-02],
         [2.60866889e-02, 3.15892388e-03, 0.00000000e+00, ...,
          0.00000000e+00, 5.15541111e-01, 3.12580569e-02],
         [2.64370290e-02, 2.86101050e-03, 0.00000000e+00, ...,
          0.00000000e+00, 5.14028590e-01, 3.15774124e-02],
...
         [2.28890015e-02, 3.22168291e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.54133705e-01, 3.12953997e-02],
         [2.32730329e-02, 2.61076201e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.61665028e-01, 3.17132839e-02],
         [2.35012987e-02, 2.33578936e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.66363607e-01, 3.19874863e-02]],

        [[2.14674734e-02, 2.72224895e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.29756620e-01, 3.12478883e-02],
         [2.16274136e-02, 2.67710858e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.30305557e-01, 3.14344781e-02],
         [2.17746256e-02, 2.63253379e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.30927608e-01, 3.16114221e-02],
         ...,
         [2.05329347e-02, 3.32736952e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.20823894e-01, 2.99901394e-02],
         [2.09658786e-02, 2.90864444e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.25130959e-01, 3.04531822e-02],
         [2.12323267e-02, 2.70951131e-02, 0.00000000e+00, ...,
          0.00000000e+00, 5.29050008e-01, 3.08084822e-02]]]])
Coordinates:
  * year        (year) int64 2050
  * Date        (Date) int32 1 91 182 274 365
  * Hour        (Hour) int32 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * technology  (technology) <U19 'Autre renouvelables' ... 'Nucleaire'

In [7]:
# Check if its is properly working : 
display(elec_mixes.sel(
    #technology='Centrales gaz', 
    year = 2050,
    Date = 365,
    Hour = 3,
))

<xarray.DataArray (technology: 9)>
array([0.02185776, 0.02608269, 0.        , 0.16383689, 0.22510046,
       0.        , 0.        , 0.53136144, 0.03176075])
Coordinates:
    year        int64 2050
    Date        int32 365
    Hour        int32 3
  * technology  (technology) <U19 'Autre renouvelables' ... 'Nucleaire'

In [8]:
# Export for next steps.
elec_mixes.to_netcdf(path = 'elec_mixes2050.nc')

In [ ]:
# Reload and make sure everything is okay : 
#elec_mixes_test = xr.open_dataset('elec_mixes.nc')
#display(elec_mixes)

### Selecting an array / single value in a more conventional format (not mandatory to make it an array/scalar)

In [12]:
# If selecting from this notebook

# Make a selection 
test = elec_mixes.sel(
    technology='Nucleaire', 
    year = 2050,
    Date = 91,
    Hour = 3,
)
# Turn it into a nice array, then values
#test = test.to_array()
test = test.values # *5+2

print(test)

0.03188951537756386


In [ ]:
# If loading from the netcdf file : 

# Make a selection 
test = elec_mixes.sel(
    technology='Centrales gaz', 
    year = 2050,
    Date = 365,
    Hour = 3,
)
# Turn it into a nice array, then values
test = test.to_array()
test = test.values[0] # *5+2

print(test)